<a href="https://colab.research.google.com/github/TattooeDeer/2DSPP/blob/master/Teacher_training_modif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install hdbscan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.2 MB 7.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp38-cp38-linux_x86_64.whl size=2700820 sha256=dbb4019a2da77093eb8a882aa74d04147a0ae14ef30d8f1b7aa5606863cb99e3
  Stored in directory: /root/.cache/pip/wheels/76/06/48/527e038689c581cc9e519c73840efdc7473805149e55bd7ffd
Successfully built hdbscan


In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.applications import VGG16

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

import hdbscan

from sklearn.cluster import KMeans, Birch
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from tensorflow.keras.utils import to_categorical
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import ParameterGrid

import itertools

SEED = 10

tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()


In [ ]:

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()


# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")



y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

170498071/170498071 [==============================] - 4s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [ ]:

porc_unlabel = 0.99
x_label, x_unlabel, y_label, y_unlabel = train_test_split(x_train, y_train, test_size=porc_unlabel, random_state=10, stratify=y_train)




In [ ]:
y_fake_label = np.zeros(shape=(len(y_unlabel), 10))

print(x_test.shape, y_test.shape)
print(x_label.shape, y_label.shape)
print(x_unlabel.shape, y_fake_label.shape)

(10000, 32, 32, 3) (10000, 10)
(500, 32, 32, 3) (500, 10)
(49500, 32, 32, 3) (49500, 10)


In [ ]:

# uno etiquetados con no etiquetados

# combino etiquetados con los no 
if x_unlabel.shape[0]>0:
  x_train = np.concatenate((x_label, x_unlabel))
  y_train = np.concatenate((y_label, y_fake_label))
  y_real = np.concatenate((y_label, y_unlabel))
else:
  x_train = x_label
  y_train = y_label








In [ ]:


cnn_input = tf.keras.Input(shape=(32, 32, 3))
upsample = tf.keras.layers.UpSampling2D()(cnn_input)
vgg16 = VGG16(include_top=False, weights='imagenet', 
              pooling='avg', input_shape=(64,64,3))(upsample)





model_cnn = keras.models.Model(cnn_input, vgg16)

model_cnn.summary()

58889256/58889256 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 64, 64, 3)        0         
 )                                                               
                                                                 
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def decay(epoch):
  """ This method create the alpha"""
  # returning a very small constant learning rate
  return 0.001 / (1 + 1 * 30)

In [ ]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()


classifier_input = tf.keras.Input(shape=(512,))
### ACÁ AGREGAR MAS CAPAS QUE PUDIERAN SER RELEVANTES, SOLO HAY UNA CAPA DE 32
classifier_dense2 = tf.keras.layers.Dense(32, activation='relu', kernel_initializer='he_uniform')(classifier_input)
classifier_dropout2 = tf.keras.layers.Dropout(0.2)(classifier_dense2)
classifier_dense3 = tf.keras.layers.Dense(len(y_train[0]), activation="softmax")(classifier_dropout2)
classifier = tf.keras.Model(classifier_input, classifier_dense3, name="classifier")

classifier.summary()
cnn_classif_input = tf.keras.Input(shape=(32, 32, 3))
cnn_series = model_cnn(cnn_classif_input)
classif_series = classifier(cnn_series)


final_classif = tf.keras.Model(cnn_classif_input, classif_series, name="CnnClf")


def my_loss(y_truee, y_pred):
    cce = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    l1 = cce(y_truee, y_pred) 
    mask = tf.reduce_sum(y_truee,axis=1)
    mloss = tf.multiply(mask,l1)
    mloss = tf.reduce_sum(mloss/tf.math.maximum(tf.reduce_sum(mask),1))
    return mloss

final_classif.compile(
loss = my_loss,
optimizer= "adam",
  metrics=['accuracy'])

callback = []
callback += [tf.keras.callbacks.LearningRateScheduler(decay, verbose=1)]
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
callback += [es]


final_classif.fit(x=x_train, y=y_train,
  batch_size=128,
  validation_data=(x_test , y_test),
  epochs=2, shuffle=True,
  callbacks=callback,
  verbose=1
  )


Model: "classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 512)]             0         
                                                                 
 dense (Dense)               (None, 32)                16416     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                330       
                                                                 
Total params: 16,746
Trainable params: 16,746
Non-trainable params: 0
_________________________________________________________________

Epoch 1: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 1/2
 27/391 [=>............................] - ETA: 2:08:51 - loss: 1.7379 - accuracy: 0.0888

------------

# Clustering

Altoritmos de Clustering a probar:
- KMeans: n_clusters
- HDBSCAN: ?
- BIRCH
- K-Medoids

In [ ]:
metrics = ['silhouette_score', 'CH', 'DB']

In [ ]:
features = model_cnn.predict(x_train)
features_test =  model_cnn.predict(x_test)

In [ ]:
## Example code
#Sil = silhouette_score(X, labels)
#CH = calinski_harabasz_score(X, labels)
#DB = davies_bouldin_score(X, labels)

In [ ]:
def eval_cluster(train_data, test_data, pred_labels_train, pred_labels_test, 
                 metrics = list(zip(metrics,  [silhouette_score, calinski_harabasz_score, davies_bouldin_score]))):
  result_dict = {'train_data': {}, 'test_data': {}}
  #result_dict['silhouette_score'] = silhouette_score(X, labels)
  #result_dict['CH'] = calinski_harabasz_score(X, labels)
  #result_dict['DB'] = davies_bouldin_score(X, labels)

  # Cartesian Product over the name of the data to evaluate (train/test), the data itself and the metric to evaluate
  sample_metric_combination = [element for element in itertools.product(zip(
                                                                          ['train_data', 'test_data'],
                                                                          [train_data, test_data],
                                                                          [pred_labels_train, pred_labels_test]),
                                                                        metrics)]

  for (sample_name, sample_data, predicted_labels), (metric_name, metric_function) in sample_metric_combination:
    result_dict[sample_name][metric_name] = metric_function(sample_data, predicted_labels)
  return result_dict

  

In [ ]:
df_best_results = pd.DataFrame(columns = pd.MultiIndex.from_product([['train_data', 'test_data'], metrics]),
                               index = pd.Index(['kmeans', 'birch', 'kmedoids', 'hdbscan'], name  = 'model_name'))
df_best_results['model'] = np.nan

### KMeans



n_clusters:
- [1,3,5] * N° clases originales



In [ ]:
kmeans_params = ParameterGrid({'n_clusters': [k*y_train.shape[1] for k in [1, 2, 5]]})
best_silhouette_kmeans = -1

for param in kmeans_params:
  k_means = KMeans(**param,random_state=SEED).fit(features)
  predict_train = k_means.predict(features)  
  predict_test =  k_means.predict(features_test) 
  results = eval_cluster(train_data = features, test_data = features_test, pred_labels_train = predict_train, pred_labels_test = predict_test)  
  
  for sample in ['train_data', 'test_data']:
    for metric in metrics:
      print(f'{sample} - {metric}: {results[sample][metric]}')
  
    if (sample == 'test_data') and (results['test_data']['silhouette_score'] > best_silhouette_kmeans): # using the silhouette score as the main metric
      df_best_results.loc['kmeans', 'model'] = k_means # Only the best model is saved
      for metric in metrics: 
        df_best_results.loc['kmeans', ('test_data', metric)] = results[sample][metric] # Only update metrics if its the best model so far
        df_best_results.loc['kmeans', ('train_data', metric)] = results[sample][metric]
      best_silhouette_kmeans = results['test_data']['silhouette_score']

  #y_kmeans_onehot = to_categorical(predict2, num_classes=k*len(y_train[0])) # pseudo - etiquetas 
  #y_kmeans_onehot_test = to_categorical(predict_test, num_classes=k*len(y_train[0]))# pseudo - etiquetas  test 

train_data - silhouette_score: 0.22556951642036438
train_data - CH: 88618.16914808439
train_data - DB: 1.2241579082653808
test_data - silhouette_score: 0.22456005215644836
test_data - CH: 17280.958927438987
test_data - DB: 1.2313653971647918
train_data - silhouette_score: 0.15953095257282257
train_data - CH: 56896.50863840548
train_data - DB: 1.4756291984487713
test_data - silhouette_score: 0.1580612063407898
test_data - CH: 11119.51588933344
test_data - DB: 1.4646383282156663
train_data - silhouette_score: 0.11288251727819443
train_data - CH: 30559.516897189784
train_data - DB: 1.6407143071743724
test_data - silhouette_score: 0.10980234295129776
test_data - CH: 5965.851724277927
test_data - DB: 1.6353108115088255


### BIRCH

In [ ]:
birch_params = ParameterGrid({'n_clusters': [k*y_train.shape[1] for k in [1, 2, 5]], 'threshold':[0.1, 0.5, 1.0, 2.0]})
best_silhouette_birch = -1

for param in birch_params:
  birch = Birch(**param).fit(features)
  predict_train = birch.predict(features)  
  predict_test =  birch.predict(features_test) 
  results = eval_cluster(train_data = features, test_data = features_test, pred_labels_train = predict_train, pred_labels_test = predict_test)  
  print(param)
  for sample in ['train_data', 'test_data']:
    for metric in metrics:
      print(f'{sample} - {metric}: {results[sample][metric]}')
  
    if (sample == 'test_data') and (results['test_data']['silhouette_score'] > best_silhouette_birch): # using the silhouette score as the main metric
      df_best_results.loc['birch', 'model'] = birch # Only the best model is saved
      for metric in metrics: 
        df_best_results.loc['birch', ('test_data', metric)] = results[sample][metric] # Only update metrics if its the best model so far
        df_best_results.loc['birch', ('train_data', metric)] = results[sample][metric]
      best_silhouette_birch = results['test_data']['silhouette_score']


### HDBSCAN

In [ ]:
import hdbscan
import joblib

In [ ]:
dbscan = hdbscan.HDBSCAN(algorithm='best', alpha=1.0, approx_min_span_tree=True,
    gen_min_span_tree=False, leaf_size=40,
    metric='euclidean', min_cluster_size=5, min_samples=None, p=None)

In [ ]:
hdbscan_params = ParameterGrid({'min_cluster_size': [5, 10, 15, 30, 60], 'min_samples':[1, 5, 10, 15, 30],
                                'cluster_selection_epsilon': [0, 0.3, 0.5, 0.7], 'cluster_selection_method': ['leaf', 'eom']})
best_silhouette_hdbscan = -1

for param in hdbscan_params:
  dbscan = hdbscan.HDBSCAN(**param).fit(features)
  predict_train = hdbscan.approximate_predict(dbscan, features)
  predict_test =  hdbscan.approximate_predict(dbscan, features_test)
  results = eval_cluster(train_data = features, test_data = features_test, pred_labels_train = predict_train, pred_labels_test = predict_test)  
  
  for sample in ['train_data', 'test_data']:
    for metric in metrics:
      print(f'{sample} - {metric}: {results[sample][metric]}')
  
    if (sample == 'test_data') and (results['test_data']['silhouette_score'] > best_silhouette_hdbscan): # using the silhouette score as the main metric
      df_best_results.loc['hdbscan', 'model'] = dbscan # Only the best model is saved
      for metric in metrics: 
        df_best_results.loc['hdbscan', ('test_data', metric)] = results[sample][metric] # Only update metrics if its the best model so far
        df_best_results.loc['hdbscan', ('train_data', metric)] = results[sample][metric]
      best_silhouette_hdbscan = results['test_data']['silhouette_score']


In [ ]:
te_df_best_results

train_data                  test_data           model
           silhouette_score   CH   DB silhouette_score   CH   DB      
model_name                                                            
kmeans                  NaN  NaN  NaN              NaN  NaN  NaN   NaN
birch                   NaN  NaN  NaN              NaN  NaN  NaN   NaN
kmedoids                NaN  NaN  NaN              NaN  NaN  NaN   NaN
hdbscan                 NaN  NaN  NaN              NaN  NaN  NaN   NaN

In [ ]:

## GAUSSIAN MIXTURE
k_gaussian = GaussianMixture(k*len(y_train[0]), random_state=10).fit(features)
predict2 = k_gaussian.predict_proba(features)


In [ ]:
y_kmeans_onehot.shape


(50000, 50)